In [602]:
import pandas as pd

ball_by_ball_data = pd.read_csv("C:/Users/Pranav Koka/Downloads/Ipl_stats/deliveries.csv")
matches_data = pd.read_csv("C:/Users/Pranav Koka/Downloads/Ipl_stats/matches.csv")

In [603]:
matches_data['match_id'] = matches_data['id']
matches_data.drop('id', axis = 1, inplace=True)

In [604]:
data = pd.merge(ball_by_ball_data, matches_data, on='match_id', how='left')
data

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1370353,2,Chennai Super Kings,Gujarat Titans,14,2,S Dube,MM Sharma,RA Jadeja,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243813,1370353,2,Chennai Super Kings,Gujarat Titans,14,3,RA Jadeja,MM Sharma,S Dube,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243814,1370353,2,Chennai Super Kings,Gujarat Titans,14,4,S Dube,MM Sharma,RA Jadeja,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243815,1370353,2,Chennai Super Kings,Gujarat Titans,14,5,RA Jadeja,MM Sharma,S Dube,6,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker


In [605]:
data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2'],
      dtype='object')

In [606]:
#ensuring that wides, noballs, and penalties don't count as a ball
for i in range(len(data)):
    if data.extras_type[i] in ['wides', 'noballs', 'penalty']:
        k = True
        j = i+1
        # Ensure j does not exceed DataFrame length
        while k and j < len(data):
            if data.ball[j] > data.ball[i]:
                data.loc[j, 'ball'] = data.ball[j] - 1
                j += 1
            else:
                k = False
        data.loc[i, 'ball'] = data.ball[i] - 1


In [607]:
#get the cumulative innings total runs
innings_total_list = []
i = 0
x = True
while i < len(data):
    innings_total = data.total_runs[i]
    innings_total_list.append(innings_total)
    i = i+1
    while i < len(data) and data.match_id[i] == data.match_id[i-1] and data.inning[i] == data.inning[i-1]:
        innings_total = innings_total + data.total_runs[i]
        innings_total_list.append(innings_total)
        i = i+1

In [608]:
data['cumulative_innings_total'] = innings_total_list 
data['cumulative_balls'] = data['over']*6 + data['ball']

In [609]:
#get the total runs scored in that innings
data['innings_total_runs'] = data.groupby(['match_id', 'inning'])['cumulative_innings_total'].transform('max')

#get the total balls bowled in that innings
data['innings_total_balls'] = data.groupby(['match_id', 'inning'])['cumulative_balls'].transform('max')

In [610]:
data['match_total_runs'] = data['target_runs'] - 1 + data['target_runs'] - 1 - data['result_margin']

In [611]:
#get the first row in each innings and then add the total balls in each innings to get the total balls bowled in the match
data_first_rows = data.groupby(['match_id', 'inning']).first().reset_index()
data_first_rows['match_total_balls'] = data_first_rows.groupby('match_id')['innings_total_balls'].transform('sum')
to_merge_df = data_first_rows[['match_id', 'match_total_balls']].drop_duplicates()

In [612]:
data = pd.merge(data, to_merge_df, on='match_id', how='left')

In [613]:
data[130:180]

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,super_over,method,umpire1,umpire2,cumulative_innings_total,cumulative_balls,innings_total_runs,innings_total_balls,match_total_runs,match_total_balls
130,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,6,W Jaffer,AB Dinda,R Dravid,0,...,N,NaN,Asad Rauf,RE Koertzen,4,6,82,91,304.0,211
131,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,I Sharma,W Jaffer,0,...,N,NaN,Asad Rauf,RE Koertzen,4,7,82,91,304.0,211
132,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,V Kohli,I Sharma,W Jaffer,0,...,N,NaN,Asad Rauf,RE Koertzen,4,8,82,91,304.0,211
133,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,V Kohli,I Sharma,W Jaffer,0,...,N,NaN,Asad Rauf,RE Koertzen,8,9,82,91,304.0,211
134,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,V Kohli,I Sharma,W Jaffer,1,...,N,NaN,Asad Rauf,RE Koertzen,9,10,82,91,304.0,211
135,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,W Jaffer,I Sharma,V Kohli,0,...,N,NaN,Asad Rauf,RE Koertzen,9,11,82,91,304.0,211
136,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,W Jaffer,I Sharma,V Kohli,0,...,N,NaN,Asad Rauf,RE Koertzen,9,12,82,91,304.0,211
137,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,V Kohli,AB Dinda,W Jaffer,0,...,N,NaN,Asad Rauf,RE Koertzen,9,13,82,91,304.0,211
138,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,V Kohli,AB Dinda,W Jaffer,0,...,N,NaN,Asad Rauf,RE Koertzen,9,14,82,91,304.0,211
139,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,JH Kallis,AB Dinda,W Jaffer,1,...,N,NaN,Asad Rauf,RE Koertzen,10,15,82,91,304.0,211


In [614]:
data['batter_total_runs'] = data.groupby(['match_id','batter'])['batsman_runs'].cumsum()
data['batter_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'penalty')].groupby(['match_id','batter']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['batter_total_balls'] = data.groupby(['match_id', 'batter'])['batter_total_balls'].transform(lambda x: x.ffill())
#if the first ball that the batter faced was a wide then there is nothing to ffill so we fill these with 0
data['batter_total_balls'] = data['batter_total_balls'].fillna(0)

In [615]:
data['bowler_total_runs'] = data[(data['extras_type'] != 'byes') & (data['extras_type'] != 'legbyes') & (data['extras_type'] != 'penalty')].groupby(['match_id', 'bowler'])['total_runs'].cumsum()
data['bowler_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'noballs') & (data['extras_type'] != 'penalty')].groupby(['match_id','bowler']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['bowler_total_runs'] = data.groupby(['match_id', 'bowler'])['bowler_total_runs'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a bye then there is nothing to ffill so we fill these with 0
data['bowler_total_runs'] = data['bowler_total_runs'].fillna(0)
#will forward fill the last valid value to the nan value
data['bowler_total_balls'] = data.groupby(['match_id', 'bowler'])['bowler_total_balls'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a noball then there is nothing to ffill so we fill these with 0
data['bowler_total_balls'] = data['bowler_total_balls'].fillna(0)

In [616]:
data['bowler_economy'] = data['bowler_total_runs']/data['bowler_total_balls'] * 6

In [617]:
data['batter_SR'] = data['batter_total_runs']/data['batter_total_balls'] * 100

In [618]:
data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2',
       'cumulative_innings_total', 'cumulative_balls', 'innings_total_runs',
       'innings_total_balls', 'match_total_runs', 'match_total_balls',
       'batter_total_runs', 'batter_total_balls', 'bowler_total_runs',
       'bowler_total_balls', 'bowler_economy', 'batter_SR'],
      dtype='object')

In [619]:
data.season.unique()

array(['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020/21', '2021', '2022',
       '2023'], dtype=object)

In [620]:
data_2008 = data[data['season'] =='2007/08']
data_2009 = data[data['season'] =='2009']
data_2010 = data[data['season'] =='2009/10']
data_2011 = data[data['season'] =='2011']
data_2012 = data[data['season'] =='2012']
data_2013 = data[data['season'] =='2013']
data_2014 = data[data['season'] =='2014']
data_2015 = data[data['season'] =='2015']
data_2016 = data[data['season'] =='2016']
data_2017 = data[data['season'] =='2017']
data_2018 = data[data['season'] =='2018']
data_2019 = data[data['season'] =='2019']
data_2020 = data[data['season'] =='2020/21']
data_2021 = data[data['season'] =='2021']
data_2022 = data[data['season'] =='2022']
data_2023 = data[data['season'] =='2023']

In [621]:
xR_data = data

In [624]:
xR_data.to_csv("C:/Users/Pranav Koka/Downloads/Ipl_stats/xR_data.csv", index=False)